In [5]:
from pyvi import ViTokenizer
import pandas as pd
import numpy as np
import random
import os
import re

# helper functions

In [8]:
def preprocess_with_segmentation (element):
    # Replace newlines, tabs, and redundant spaces with a single space
    processed_element = re.sub(r'[\n\t\s]+', ' ', element)
    
    # Strip leading and trailing spaces
    processed_element = processed_element.strip()
    
    # Tokenize the text using ViTokenizer
    segmented_element = ViTokenizer.tokenize(processed_element)
    
    return segmented_element

In [9]:
def preprocess_no_segmentation(element):
    # Replace newlines, tabs, and redundant spaces with a single space
    processed_element = re.sub(r'[\n\t\s]+', ' ', element)
    
    # Strip leading and trailing spaces
    processed_element = processed_element.strip()
    
    return processed_element

# prepare corpus and test set

In [10]:
columns_to_drop = ['index', 'question', 'extractive answer', 'abstractive answer', 'yes/no']

In [11]:
test_columns_to_drop = ['index', 'extractive answer', 'abstractive answer', 'yes/no']

## word-segmented format 

In [37]:
df_train = pd.read_csv("[R2GQA]-data-full/train.csv")
df_valid = pd.read_csv("[R2GQA]-data-full/val.csv")
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")

corpus = pd.concat([df_train, df_valid, df_test], axis=0)
print('corpus shape:', corpus.shape)
corpus.head(3)

corpus shape: (9758, 8)


,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,8276,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG,Sinh viên dự bị không trở thành sinh viên chín...,02 học kỳ liên tiếp,Sinh viên dự bị không trở thành sinh viên chín...,NaN
1,8543,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,theo đề xuất hợp lý của bộ phận chuyên môn,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,NaN
2,7152,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...,N3,Sinh viên CT CLC phải đạt tiếng Nhật N3 thì mớ...,NaN


In [39]:
corpus.drop(columns=columns_to_drop, inplace=True)
print('corpus shape:', corpus.shape)
corpus.head(3)

corpus shape: (9758, 3)


,context,article,document
0,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...
2,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO


In [40]:
corpus.drop_duplicates(inplace=True)
print('corpus shape:', corpus.shape)

corpus shape: (294, 3)


In [42]:
corpus = corpus.map(preprocess_with_segmentation)
corpus.head(3)

,context,article,document
0,Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_...,Điều 9 . Tuyển bổ_sung và loại ra khỏi chương_...,QUY_ĐỊNH ĐÀO_TẠO CHƯƠNG_TRÌNH TÀI_NĂNG
1,Điều 4 . Kiểm_tra xếp lớp đầu khóa cho sinh_vi...,Điều 4 . Kiểm_tra xếp lớp đầu khóa cho sinh_vi...,QUY_ĐỊNH ĐÀO_TẠO NGOẠI_NGỮ ĐỐI_VỚI HỆ ĐẠI_HỌC ...
2,Điều 5 . Chương_trình đào_tạo CT CLC được xâ...,Điều 5 . Chương_trình đào_tạo,QUY_ĐỊNH ĐÀO_TẠO CHƯƠNG_TRÌNH CHẤT_LƯỢNG CAO


In [43]:
os.makedirs('./corpus', exist_ok=True)
corpus.to_csv('corpus/corpus-seg.csv', index=False)

In [46]:
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")
print(df_test.shape)
df_test.head(3)

(976, 8)


,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,4206,"Điều 8. Xây dựng, thẩm định học liệu điện tử\...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...,Trường có thể tổ chức thẩm định lại trong trườ...,Trường có thể tổ chức thẩm định lại trong trườ...,1.0
1,960,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa n...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...,Các tài liệu trong hồ sơ đề nghị chỉnh sửa văn...,Việc nộp bản sao giấy khai sinh cho hồ sơ đề n...,NaN
2,2151,Điều 23. Điểm Miễn\n1. Điểm BL\n- Sin...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...,Nếu trùng thời gian thi cuối kỳ: sinh viên đượ...,Quy định về điểm M trong trường hợp sinh viên ...,NaN


In [48]:
df_test.drop(columns=test_columns_to_drop, inplace=True)
print(df_test.shape)
df_test.head(3)

(976, 4)


,context,article,document,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử\...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa n...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn\n1. Điểm BL\n- Sin...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...


In [50]:
df_test = df_test.map(preprocess_with_segmentation)
df_test.head(3)

,context,article,document,question
0,"Điều 8 . Xây_dựng , thẩm_định học liệu điện_tử...","Điều 8 . Xây_dựng , thẩm_định học liệu điện_tử",QUY_ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG_THỨC TRỰC_TUYẾ...,Học_liệu điện_tử sau khi được thông_qua bởi ĐV...
1,"Điều 20 . Hồ_sơ , trình_tự , thủ_tục chỉnh_sửa...","Điều 20 . Hồ_sơ , trình_tự , thủ_tục chỉnh_sửa...","QUY_CHẾ Văn_bằng , chứng_chỉ của Trường Đại_họ...",Có_thể nộp bản_sao giấy khai_sinh cho hồ_sơ đề...
2,Điều 23 . Điểm_Miễn 1 . Điểm_BL - Sinh_viê...,Điều 23 . Điểm_Miễn,QUY_CHẾ ĐÀO_TẠO THEO HỌC CHẾ TÍN_CHỈ CHO HỆ ĐẠ...,Quy_định về điểm M trong trường_hợp sinh_viên ...


In [52]:
os.makedirs('./corpus', exist_ok=True)
df_test.to_csv('corpus/corpus-question-test-map-seg.csv', index=False)

## no word-segmented format 

In [12]:
df_train = pd.read_csv("[R2GQA]-data-full/train.csv")
df_valid = pd.read_csv("[R2GQA]-data-full/val.csv")
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")

corpus = pd.concat([df_train, df_valid, df_test], axis=0)
print('corpus shape:', corpus.shape)
corpus.head(3)

corpus shape: (9758, 8)


,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,8276,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG,Sinh viên dự bị không trở thành sinh viên chín...,02 học kỳ liên tiếp,Sinh viên dự bị không trở thành sinh viên chín...,NaN
1,8543,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,theo đề xuất hợp lý của bộ phận chuyên môn,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,NaN
2,7152,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...,N3,Sinh viên CT CLC phải đạt tiếng Nhật N3 thì mớ...,NaN


In [13]:
corpus.drop(columns=columns_to_drop, inplace=True)
print('corpus shape:', corpus.shape)
corpus.head(3)

corpus shape: (9758, 3)


,context,article,document
0,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...
2,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO


In [14]:
corpus.drop_duplicates(inplace=True)
print('corpus shape:', corpus.shape)

corpus shape: (294, 3)


In [15]:
corpus = corpus.map(preprocess_no_segmentation)
corpus.head(3)

,context,article,document
0,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...
2,Điều 5. Chương trình đào tạo CT CLC được xây d...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO


In [16]:
os.makedirs('./corpus', exist_ok=True)
corpus.to_csv('corpus/corpus.csv', index=False)

In [19]:
df_test = pd.read_csv("[R2GQA]-data-full/test.csv")
print(df_test.shape)
df_test.head(3)

(976, 8)


,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,4206,"Điều 8. Xây dựng, thẩm định học liệu điện tử\...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...,Trường có thể tổ chức thẩm định lại trong trườ...,Trường có thể tổ chức thẩm định lại trong trườ...,1.0
1,960,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa n...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...,Các tài liệu trong hồ sơ đề nghị chỉnh sửa văn...,Việc nộp bản sao giấy khai sinh cho hồ sơ đề n...,NaN
2,2151,Điều 23. Điểm Miễn\n1. Điểm BL\n- Sin...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...,Nếu trùng thời gian thi cuối kỳ: sinh viên đượ...,Quy định về điểm M trong trường hợp sinh viên ...,NaN


In [20]:
df_test.drop(columns=test_columns_to_drop, inplace=True)
print(df_test.shape)
df_test.head(3)

(976, 4)


,context,article,document,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử\...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa n...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn\n1. Điểm BL\n- Sin...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...


In [21]:
df_test = df_test.map(preprocess_no_segmentation)
df_test.head(3)

,context,article,document,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử 1...","Điều 8. Xây dựng, thẩm định học liệu điện tử",QUY ĐỊNH DẠY VÀ HỌC THEO PHƯƠNG THỨC TRỰC TUYẾ...,Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...","QUY CHẾ Văn bằng, chứng chỉ của Trường Đại học...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn 1. Điểm BL - Sinh viên đã t...,Điều 23. Điểm Miễn,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,Quy định về điểm M trong trường hợp sinh viên ...


In [22]:
os.makedirs('./corpus', exist_ok=True)
df_test.to_csv('corpus/corpus-question-test-map.csv', index=False)